# select raw images

In [ ]:
## Construct a list of files you wish to view and/or combine.

acceptlist = False             # True if you want to accept all the files in somefiles.
contiguous = True             # True if you want to accept a contiguous subset of somefiles.
startfile, endfile = 0,9      # The first and last files in a contiquous subset.
flist = [0,3,4,5,6,7,8]        # An explicit list of the files you want to accept.

if acceptlist == True:
    files = somefiles
    for i in range(len(files)):
        print(i, files[i])
else:
    if contiguous == True:
        files = somefiles[startfile:endfile+1]
        for i in range(len(files)):
            print( i, files[i])
    else:
        files = []
        for i in range(len(flist)):
            files.append(somefiles[flist[i]])
        for i in range(len(files)):
            print( i, flist[i], files[i])

In [ ]:
## Load the images into the stack.
image, headlist, rows, cols, imed, imad = stackfits(datapath, files)

In [ ]:
## Display the image stack.
        
for i in range(len(files)):
    quickpic(image[i],files[i])

# Select and load darkfile, then make unmasked and masked dark images

In [ ]:
'''Get a dark image, calculate some statistics, make a masked image with "nans" replacing "bad" pixels, and
calculate some statistics on the masked image. In this case, "bad" is defined as pixels lying above or below
selected thresholds. For example, one might want to exclude "dead" pixels with no signal or "hot" pixels with
bias levels so high that that the pixel's dynamic range would be unacceptable.'''

whichfile = 0
highlimfactor = 20.
lowlimfactor = 2.

darkname = darkfiles[whichfile]    # Select a dark image.
dname = os.path.join(darkpath,darkname)
darkhdulist = fits.open(dname)
darkhdulist.info()
print('')

darkimage = darkhdulist[0].data * 1.0          # Read in the image and convert to float.
darkheader = darkhdulist[0].header.copy()      # Make a header for the darkimage, a copy of the original file header.

# Calculate median and std of raw bias image.
darkmedian = np.nanmedian(darkimage)
darkstd = np.nanstd(darkimage)
darkmad = mad_std(darkimage)
darkmin = np.nanmin(darkimage)
darkmax = np.nanmax(darkimage)
print('darkmedian=', darkmedian, '  darkstd=', darkstd,'   darkmad=',darkmad, '  darkmin=', darkmin,'  darkmax=', darkmax)

# Set upper and lower thresholds for "good" pixels and make masks.
dupperlimit = highlimfactor * darkmedian   # Select an upper limit for "good" pixels.
dlowerlimit = darkmedian/lowlimfactor     # Select a lower limit for "good" pixels.
print('lower limit =',dlowerlimit, '  upper limit =', dupperlimit)
highdarkmask = np.where(darkimage > dupperlimit)
lowdarkmask = np.where(darkimage < dlowerlimit)

# Make masked images with 'nans' in masked pixels.
mdarkimage = darkimage.copy()
mdarkimage[highdarkmask] = np.nan
mdarkimage[lowdarkmask] = np.nan

# Re-compute median and std for masked image.
# NOTE: mad_std doesn't seem to work if there are nans in the image.
mdmedian = np.nanmedian(mdarkimage)
mdstd = np.nanstd(mdarkimage)
mdmad = mad_std(mdarkimage,ignore_nan=True)
mdmin = np.nanmin(mdarkimage)
mdmax = np.nanmax(mdarkimage)
print('mask median =',mdmedian,' mask std =',mdstd,' mask mad',mdmad,' mask min =',mdmin,' mask max =',mdmax)
print('low pixels =', len(lowdarkmask[0]),'   high pixels =', len(highdarkmask[0]),)

## Use this if you want to print the header.
# darkheader

# list high or low pixels

In [ ]:
# ## List the low and/or high pixels in the darkmasks.

 low = False
 high = True
 if low == True:
     print('Number of low pixels =',len(lowdarkmask[0]))
     print('')
     for i in range(len(lowdarkmask[0])):
         print(i, lowdarkmask[0][i], lowdarkmask[1][i])
 if high == True:
     print('Number of high pixels =',len(highdarkmask[0]))
     print('')
     for i in range(len(highdarkmask[0])):
         print(i, highdarkmask[0][i], highdarkmask[1][i])

# Select a biasfile and make unmasked and masked bias images

In [ ]:
# Get a bias image, calculate some statistics, and make a masked image with "nans" replacing "bad" pixels.

biasname = biasfiles[0]    # Select a bias image.

hblimfactor = 2.
lblimfactor = 2.

bname = os.path.join(biaspath,biasname)
biashdulist = fits.open(bname)
biashdulist.info()
print('')

biasimage = biashdulist[0].data * 1.0    # Read in the image and convert to float.
biasheader = biashdulist[0].header.copy()
print('')

# Calculate median and std of raw bias image.
biasmedian = np.nanmedian(biasimage)
biasstd = np.nanstd(biasimage)
biasmad = mad_std(biasimage,ignore_nan=True)
biasmin = np.nanmin(biasimage)
biasmax = np.nanmax(biasimage)
print('median=', biasmedian, '  std=', biasstd, '  min=', np.nanmin(biasimage),'  max=', np.nanmax(biasimage))

# Set upper and lower thresholds for "good" pixels and make masks.
bupperlimit = hblimfactor * biasmedian   # Select an upper limit for "good" pixels.
blowerlimit = biasmedian/lblimfactor     # Select a lower limit for "good" pixels.
print('lower limit =',blowerlimit, '  upper limit =', bupperlimit)
highbiasmask = np.where(biasimage > bupperlimit)
lowbiasmask = np.where(biasimage < blowerlimit)
print('high pixels =', len(highbiasmask[0]), '    low pixels =', len(lowbiasmask[0]))

# Make masked images with 'nans' in masked pixels.
mbiasimage = biasimage.copy()
mbiasimage[highbiasmask] = np.nan
mbiasimage[lowbiasmask] = np.nan

# Re-compute median and std for masked image.
mbmedian = np.nanmedian(mbiasimage)
mbstd = np.nanstd(mbiasimage)
mbmad = mad_std(mbiasimage,ignore_nan=True)
mbmin = np.nanmin(mbiasimage)
mbmax = np.nanmax(mbiasimage)
print('mask median =',mbmedian,' mask std =', mbstd,'mask mad =',mbmad, ' mask min =',mbmin,' masked max =',mbmax)

In [ ]:
# ## List the low and/or high pixels in the biasmasks.

 low = True
 high = True
 if low == True:
     print('Number of low pixels =',len(lowbiasmask[0]))
     for i in range(len(lowbiasmask[0])):
         print(i, lowbiasmask[0][i], lowbiasmask[1][i])
     print('')
 if high == True:
     print('Number of high pixels =',len(highbiasmask[0]))
     for i in range(len(highbiasmask[0])):
         print(i, highbiasmask[0][i], highbiasmask[1][i])

In [ ]:
## Make list of high pixels (greater than some value) in the biasimage and their values. DIAGNOSTIC.

hbmask = np.where(biasimage > bupperlimit)
print( hbmask)
# print hbmask[0][0]
for i in range(len(hbmask[0])):
    print ('{:3}{:4}{:5}{:10}'.format(i,hbmask[0][i],hbmask[1][i],biasimage[hbmask[0][i],hbmask[1][i]]))

# Dark current image

In [ ]:
# Make a dark current image. Also make masks to screen out high and low pixels. 

dcimage = darkimage - biasimage
dcheader = darkheader.copy()
dcheader['proctype'] = 'dark current'
dcheader['filelist'] = darkname + ', ' + biasname

# Calculate some statistics

dcmin, dcmax = np.nanmin(dcimage), np.nanmax(dcimage)
print(dcmin, dcmax)
dcmedian = np.nanmedian(dcimage)
dcstd = np.nanstd(dcimage)
dcmad = mad_std(dcimage)
print(dcmedian, dcstd, dcmad)

dclowerlimit = dcmedian - 5 * dcmad
dcupperlimit = dcmedian + 100 * dcmad
# dclowerlimit = dcmedian - 5 * dcmad
# dcupperlimit = dcmedian + 50 * dcmad
lowdcmask = np.where(dcimage <  dclowerlimit)
highdcmask = np.where(dcimage > dcupperlimit)

print('high pixels =', len(highdcmask[0]), '    low pixels =', len(lowdcmask[0]))

mdcimage = dcimage.copy()
mdcimage[highdcmask] = np.nan
mdcimage[lowdcmask] = np.nan

mdcmedian = np.nanmedian(mdcimage)
mdcstd = np.nanstd(mdcimage)
print(mdcmedian, mdcstd, np.max(mdcimage), np.min(mdcimage))

In [ ]:
# ## Above I have used dcheader = darkheader.copy(). Seemed to have worked.
# ## The header class does have a .copy() function. See it when you use the ?? magic below.
# ## Use the .copy() function when you intend to modify the header and use it in another fits object.
# dcheader??

In [ ]:
## Print lists of pixels above or below some threshold.
printlist = 'no'

print('max pixel =',np.max(dcimage))
himask = np.where(dcimage > dcupperlimit)  # Or, put in some other limit.
lomask = np.where(dcimage < dclowerlimit)  # Or, put in some other limit.

whichmask = himask

print('pixels above threshold =', len(whichmask[0]), '    threshold =', dcupperlimit)
print('')

if printlist == 'yes':
    for i in range(len(whichmask[0])):
        print ('{:3}{:5}{:5}{:10}'.format(i,whichmask[0][i],whichmask[1][i],dcimage[whichmask[0][i],whichmask[1][i]]))

In [ ]:
quickpic(dcimage)

In [ ]:
displaypic2(dcimage,'',1024,1024,istart=(dcmedian- 3* dcmad,dcmedian+3*dcmad),negative=True)

In [ ]:
## Plot some histograms of the images for diagnostic purposes.
## This uses histogram plotting routines from matplotlib.

NBINS = 200
plt.figure(figsize = ((18,8)))

dclow, dchigh = np.nanmin(dcimage), np.nanmax(dcimage)
print( 'low =', dclow, ',   high =', dchigh)
dcunderlimit, dcoverlimit = dclow, dcupperlimit
dcundermask, dcovermask = np.where(dcimage < dcunderlimit), np.where(dcimage >= dcoverlimit)
# print undermask[0]

print( 'pixels <',dcunderlimit, '=',len(dcundermask[0]))
print( 'pixels >',dcoverlimit, '=', len(dcovermask[0]))

plt.subplot(3,1,1)
plt.grid()
plt.title('dark current in  '+ darkname)
# xlim(0000.,100)
# ylim(0,200)
histogram = pylab.hist(dcimage.flatten(),NBINS,histtype='step',range=(dclow,dcoverlimit/2.))
plt.subplot(3,1,2)
histogram = pylab.hist(dcimage.flatten(),histtype='step',log=True)

# load and display a flatfile

In [ ]:
# Get a flat image, calculate some statistics, and make a masked image with "nans" replacing "bad" pixels.

whichfile = 0

hflimfactor = 2.
lflimfactor = 2.

flatname = flatfiles[whichfile]    # Select a flat image.
fname = os.path.join(flatpath,flatname)
flathdulist = fits.open(fname)
flathdulist.info()
print('')

flatimage = flathdulist[0].data * 1.0    # Read in the image and convert to float.
flatheader = flathdulist[0].header.copy()

print('')

# Calculate median and std of raw flat image.
flatmedian = np.nanmedian(flatimage)
flatstd = np.nanstd(flatimage)
flatmad = mad_std(flatimage,ignore_nan=True)
flatmin = np.nanmin(flatimage)
flatmax = np.nanmax(flatimage)
print('median=',flatmedian,'  std=',flatstd,'flatmad =',flatmad,'  min=',flatmin,'  max=', flatmax)

# Set upper and lower thresholds for "good" pixels and make masks.
fupperlimit = hflimfactor * flatmedian   # Select an upper limit for "good" pixels.
flowerlimit = flatmedian/lflimfactor     # Select a lower limit for "good" pixels.
print('lower limit =',flowerlimit, '  upper limit =', fupperlimit)
highflatmask = np.where(flatimage > fupperlimit)
lowflatmask = np.where(flatimage < flowerlimit)
print('high pixels =', len(highflatmask[0]), '    low pixels =', len(lowflatmask[0]))

# Make masked images with 'nans' in masked pixels.
mflatimage = flatimage.copy()
mflatimage[highflatmask] = np.nan
mflatimage[lowflatmask] = np.nan

# plt.colorbar()
print('highmask =',highflatmask)

# Re-compute median and std for masked image.
mfmedian = np.nanmedian(mflatimage)
mfstd = np.nanstd(mflatimage)
mfmad = mad_std(mflatimage,ignore_nan=True)
mfmin = np.nanmin(mflatimage)
mfmax = np.nanmax(mflatimage)
print('mask median =',mfmedian,' mask std =', mfstd,'mask mad =',mfmad, ' mask min =',mfmin,' mask max =',mfmax)

In [ ]:
displaypic2(flatimage,flatname,rows,cols,istart=(flatmedian- 3* flatmad,flatmedian+3*flatmad),negative=False)

# make bad pixel masks for flat image

In [ ]:
# ## Make lists of high and low pixels (more or less than 5 sigma) in the flatimage ("suspicious pixels") and their values

# # Print lists?
printlists = 'no'

multiplier = 10.

highlimit = flatmedian + multiplier*flatmad
lowlimit = flatmedian - multiplier*flatmad
hfmask = np.where(flatimage > highlimit)   # Or put in some other value.
lfmask = np.where(flatimage < lowlimit)   # Or put in some other value.
print('high threshold =',highlimit,'   low threshold =',lowlimit)
print( 'high pixels =', len(hfmask[0]), '    low pixels =', len(lfmask[0]))
print( '')
if printlists == 'yes':
    print( 'list of high pixels')
    for i in range(len(hfmask[0])):
        print( ('{:3}{:5}{:5}{:>10.3f}'.format(i,hfmask[0][i],hfmask[1][i],flatimage[hfmask[0][i],hfmask[1][i]])))
    print( '')
    print( 'low pixels')
    for i in range(len(lfmask[0])):
        print( ('{:3}{:5}{:5}{:>10.3f}'.format(i,lfmask[0][i],lfmask[1][i],flatimage[lfmask[0][i],lfmask[1][i]])))

# Plot a row and/or column

In [ ]:
rowcolplot(rawimage,files[whichimage],750,350,plotrow=True,plotcol=True,dots=True,ylims=(3000.,11000.),autoscale=False)